# Convert MedEx output to i2b2 format

In [1]:
import os

In [2]:
output_no_drools = os.listdir("./output_no_drools/") #folder containing medex-processed test files (drools rule engine was not used)
output_with_drools = os.listdir("./output_with_drools/") #folder containing medex-processed test files (drools rule engine was used)

In [ ]:
os.makedirs("./output_no_drools_i2b2/")
os.makedirs("./output_with_drools_i2b2/")

Copy "train.test.released.8.17.09" folder from i2b2 to this directory

### Output without drools rule engine

In [3]:
for file in output_no_drools:
        
    lst = [] # [line index, line] for line in file
    first_char_to_line_token = {} # {first character index: (line index, token index),...}
    last_char_to_line_token = {} # {last character index: (line index, token index),...}
    t_len = 0  # counts characters per token

    with open("./output_no_drools/"+file) as r:

        x = r.readlines()
        e = enumerate(x)
        for line_id, line in e:
            lst.append([line_id, line])

    with open("./train.test.released.8.17.09/"+file) as inputfile:

        input_txt = inputfile.read().lower()
        input_lines = input_txt.split("\n")
        for l_id, l in enumerate(input_lines):  
            for t_id, t in enumerate(l.split()):  
                first_char_id = input_txt.find(t, t_len)
                last_char_id = first_char_id + len(t)
                first_char_to_line_token[first_char_id] = (l_id+1, t_id)
                last_char_to_line_token[last_char_id] = (l_id+1, t_id)
                t_len = first_char_id + len(t)

    with open("./output_no_drools_i2b2/"+file, "w") as w:
        for line_id, line in lst:
            if "|" not in line.split()[0]: # only loop over lines that contain medication mentions

                line = line.replace("\t", "|", 1)
                line = line.split("|")

                m = line[2].split("[")[0] #presumed that there is no "[" in the medication name
                first_char_id = int(line[2].split("[")[-1].split(",")[0])  
                last_char_id = int(line[2].split("[")[-1].split(",")[-1][:-1])
                idx = int(line[0])

                # medex treats tokens in the form of XXX-XXX, XXX/XXX, and XXX+XXX as two tokens 
                # which leads to inaccurate start/end indices
                while True: 
                    try:
                        first_m_line_id = first_char_to_line_token[int(first_char_id)][0]
                        first_m_token_id = first_char_to_line_token[int(first_char_id)][1]
                        break
                    except:
                        first_char_id -= 1

                while True:
                    try: 
                        last_m_line_id = last_char_to_line_token[int(last_char_id)][0]
                        last_m_token_id = last_char_to_line_token[int(last_char_id)][1]
                        break
                    except:
                        last_char_id += 1

                annotation = f'm="{m}" {first_m_line_id}:{first_m_token_id} {last_m_line_id}:{last_m_token_id}||du="nm"\n' 
                # dummy annotation

                for sent_id, sent in lst:

                    if '|' in sent.split()[0] and 'DRT' in sent and int(sent.split('|')[0]) == idx-1: 
                        # only duration mentions related to drugs are relevant!

                        dur = sent.split('|')[-4]
                        first_du_char_id = int(sent.split('|')[-2])
                        last_du_char_id = int(sent.split('|')[-1])

                        if dur in line[1]:

                            first_du_line_id = first_char_to_line_token[int(first_du_char_id)][0]
                            first_du_token_id = first_char_to_line_token[int(first_du_char_id)][1]

                            last_du_line_id = last_char_to_line_token[int(last_du_char_id)][0]
                            last_du_token_id = last_char_to_line_token[int(last_du_char_id)][1]

                            annotation = f'm="{m}" {first_m_line_id}:{first_m_token_id} {last_m_line_id}:{last_m_token_id}||du="{dur}" {first_du_line_id}:{first_du_token_id} {last_du_line_id}:{last_du_token_id}\n'

                w.write(annotation)

### Output with Drools rule engine

In [4]:
for file in output_with_drools:
    
    lst = [] # all lines in file
    first_char_to_line_token = {} # {first character index: (line index, token index),...}
    last_char_to_line_token = {} # {last character index: (line index, token index),...}
    t_len = 0  # counts characters per token
    
    with open("./output_with_drools/"+file) as r:
        
        x = r.readlines()
        e = enumerate(x)
        for line_id, line in e:
            lst.append([line_id, line])
    
    with open("./train.test.released.8.17.09/"+file) as inputfile:
        
        input_txt = inputfile.read().lower()
        input_lines = input_txt.split("\n")
        for l_id, l in enumerate(input_lines):  
            for t_id, t in enumerate(l.split()):  
                first_char_id = input_txt.find(t, t_len)
                last_char_id = first_char_id + len(t)
                first_char_to_line_token[first_char_id] = (l_id+1, t_id)
                last_char_to_line_token[last_char_id] = (l_id+1, t_id)
                t_len = first_char_id + len(t)
                    
    with open("./output_with_drools_i2b2/"+file, "w") as w:
        for line_id, line in lst:
            if "|" not in line.split()[0]: # only loop over lines that contain medication mentions
                
                line = line.replace("\t", "|", 1)
                line = line.split("|")
                
                m = line[2].split("[")[0] #presumed that there is no "[" in the medication name
                first_char_id = int(line[2].split("[")[-1].split(",")[0])  
                last_char_id = int(line[2].split("[")[-1].split(",")[-1][:-1])
                idx = int(line[0])

                # medex treats tokens in the form of XXX-XXX, XXX/XXX, and XXX+XXX as two tokens 
                # which leads to inaccurate start/end indices
                while True: 
                    try:
                        first_m_line_id = first_char_to_line_token[int(first_char_id)][0]
                        first_m_token_id = first_char_to_line_token[int(first_char_id)][1]
                        break
                    except:
                        first_char_id -= 1

                while True:
                    try: 
                        last_m_line_id = last_char_to_line_token[int(last_char_id)][0]
                        last_m_token_id = last_char_to_line_token[int(last_char_id)][1]
                        break
                    except:
                        last_char_id += 1

                annotation = f'm="{m}" {first_m_line_id}:{first_m_token_id} {last_m_line_id}:{last_m_token_id}||du="nm"\n' 
                # dummy annotation

                for sent_id, sent in lst:

                    if '|' in sent.split()[0] and 'DRT' in sent and int(sent.split('|')[0]) == idx-1: 
                        # only duration mentions related to drugs are relevant!

                        dur = sent.split('|')[-4]
                        first_du_char_id = int(sent.split('|')[-2])
                        last_du_char_id = int(sent.split('|')[-1])

                        if dur in line[1]:
                            
                            first_du_line_id = first_char_to_line_token[int(first_du_char_id)][0]
                            first_du_token_id = first_char_to_line_token[int(first_du_char_id)][1]

                            while True:
                                try:
                                    last_du_line_id = last_char_to_line_token[int(last_du_char_id)][0]
                                    last_du_token_id = last_char_to_line_token[int(last_du_char_id)][1]
                                    break
                                except:
                                    last_du_char_id += 1
                                    
                            annotation = f'm="{m}" {first_m_line_id}:{first_m_token_id} {last_m_line_id}:{last_m_token_id}||du="{dur}" {first_du_line_id}:{first_du_token_id} {last_du_line_id}:{last_du_token_id}\n'

                w.write(annotation)

# Calculate MedEx-UIMA scores

In [5]:
import re

In [ ]:
os.makedirs('./human_annotations/')

Copy all files in the i2b2 folders "annotations_ground_truth/converted.noduplicates.sorted/" (except 'bucharest_v1.py' and '827931.Alan.Aronson.NIH.NLM.a1_827931.Faisal.Farooq.Siemens.a2.Kim.m.save') and in "training_ground_truth" (except 'CHANGE.HISTORY.txt') to the "human_annotations" folder created in the previous cell.

In [6]:
annotations = os.listdir('./human_annotations/')

In [8]:
for filename in annotations:
    try:
        base = re.split(r'\D', filename)[0]
        os.rename('./human_annotations/'+filename, './human_annotations/'+base)
    except:
        pass

Manually rename any files that were skipped: Only the file indices should be kept, e.g. '1234_gold' => '1234'

### Token matches, no Drools

In [10]:
m_tp = 0
m_fn = 0
m_fp = 0

dur_tp = 0
dur_fn = 0
dur_fp = 0

for filename in output_no_drools:
    
    pred_dict = {} # predicted medication index: related predicted duration index
    gold_dict = {} # gold medication index: related gold duration index
    
    with open("./output_no_drools_i2b2/"+filename) as m, open("./human_annotations/"+filename) as h:
        
        machine_lines = m.readlines()
        human_lines = h.readlines()
        
    for line in machine_lines:
            
        m = line.split('||')[0]
        dur = line.split('||')[1]
        m_idx = ' '.join(m.split()[-2:])
            
        if dur.lower().strip() == 'du="nm"':
            pred_dict[m_idx] = 'du="nm"' 
        else:
            dur_idx = ' '.join(dur.split()[-2:])
            pred_dict[m_idx] = dur_idx
                    
    for line in human_lines:
            
        m = line.split('||')[0]
        dur = line.split('||')[1]
        m_idx = ' '.join(m.split()[-2:])
            
        if dur.lower().split() == 'du="nm"':
            gold_dict[m_idx] = 'du="nm"'
            
        else:
            dur_idx = ' '.join(dur.split()[-2:])
            gold_dict[m_idx] = dur_idx
                                
    for drug, duration in pred_dict.items(): 
        pred_drug_line = int(drug.split(':')[0])
        pred_drug_start_token = int(drug.split(':')[1].split()[0])
        pred_drug_end_token = int(drug.split(':')[-1])
        pred_drug = range(pred_drug_start_token, pred_drug_end_token+1)
        drug_cnt = 0 # turns to 1 if a corresponding drug was found in the human annotations
            
        for drug in gold_dict.keys():
            gold_drug_line = int(drug.split(':')[0])
            try:
                gold_drug_start_token = int(drug.split(':')[1].split()[0])
            except: # one exception found were drug = "217:1,217:3 217:3" instead of "217:1 217:3"
                gold_drug_start_token = int(drug.split(':')[1].split(',')[0])       
            gold_drug_end_token = int(drug.split(':')[-1])
            gold_drug = range(gold_drug_start_token, gold_drug_end_token+1)
            overlap = set(pred_drug) & set(gold_drug)
                
            if pred_drug_line == gold_drug_line and overlap: 
                # corresponding drug mention found, assuming that drug mentions are not split over lines
                
                drug_cnt += 1
                m_tp += len(overlap)
                
                if pred_drug_end_token > gold_drug_end_token:
                    m_fp += pred_drug_end_token - gold_drug_end_token
                if pred_drug_end_token < gold_drug_end_token:
                    m_fn += gold_drug_end_token - pred_drug_end_token
                if pred_drug_start_token < gold_drug_start_token:
                    m_fp += gold_drug_start_token - pred_drug_start_token
                if pred_drug_start_token > gold_drug_start_token:
                    m_fn += pred_drug_start_token - gold_drug_start_token
                    
                break
                    
        if drug_cnt == 0:
            m_fp += pred_drug_end_token - pred_drug_start_token + 1
                
        if duration.strip() != 'du="nm"':
            pred_start_line = int(duration.split(':')[0])
            pred_end_line = int(duration.split(':')[1].split()[-1])
            pred_dur_start_token = int(duration.split(':')[1].split()[0])
            pred_dur_end_token = int(duration.split(':')[-1])
            dur_cnt = 0 # turns to 1 if a corresponding duration was found in the human annotations
            
            if pred_start_line == pred_end_line:
                pred_dur = range(pred_dur_start_token, pred_dur_end_token+1)
                
                for duration in gold_dict.values():
                    if duration.strip() != 'du="nm"':
                        gold_dur_line = int(duration.split(':')[0])
                        gold_dur_start_token = int(duration.split(':')[1].split()[0])
                        gold_dur_end_token = int(duration.split(':')[-1])
                        gold_dur = range(gold_dur_start_token, gold_dur_end_token+1)
                        overlap = set(pred_dur) & set(gold_dur)
                
                        if pred_start_line == gold_dur_line and overlap:
                            dur_cnt += 1
                            dur_tp += len(overlap)
                            
                            if pred_dur_end_token > gold_dur_end_token:
                                dur_fp += pred_dur_end_token - gold_dur_end_token
                            if pred_dur_end_token < gold_dur_end_token:
                                dur_fn += gold_dur_end_token - pred_dur_end_token
                            if pred_dur_start_token < gold_dur_start_token:
                                dur_fp += gold_dur_start_token - pred_dur_start_token
                            if pred_dur_start_token > gold_dur_start_token:
                                dur_fn += pred_dur_start_token - gold_dur_start_token
                            break
                
                if dur_cnt == 0:
                    dur_fp += pred_dur_end_token - pred_dur_start_token + 1
                            
            else:  
                with open('./input/'+filename) as f:
                    x = f.readlines()
                    for idx, line in enumerate(x):
                        if idx+1 == int(pred_start_line):
                            line_len = len(line.split())
                            break
                            
                for duration in gold_dict.values():
                     if duration.strip() != 'du="nm"':
                        gold_start_line = int(duration.split(':')[0])
                        gold_end_line = int(duration.split(':')[1].split()[-1])
                        gold_dur_start_token = int(duration.split(':')[1].split()[0])
                        gold_dur_end_token = int(duration.split(':')[-1])
                                    
                        if pred_start_line == gold_start_line and pred_end_line == gold_end_line:
                            first_line_overlap = set(range(pred_dur_start_token, line_len)) & set(range(gold_dur_start_token, line_len))
                            second_line_overlap = set(range(0, pred_dur_end_token+1)) & set(range(0, gold_dur_end_token+1))
                            
                            if first_line_overlap or second_line_overlap:
                                dur_cnt += 1
                                dur_tp += len(first_line_overlap)+len(second_line_overlap)
                                
                                if pred_dur_end_token > gold_dur_end_token:
                                    dur_fp += pred_dur_end_token - gold_dur_end_token
                                if pred_dur_end_token < gold_dur_end_token:
                                    dur_fn += gold_dur_end_token - pred_dur_end_token
                                if pred_dur_start_token < gold_dur_start_token:
                                    dur_fp += gold_dur_start_token - pred_dur_start_token
                                if pred_dur_start_token > gold_dur_start_token:
                                    dur_fn += pred_dur_start_token - gold_dur_start_token
                                break 
                        
                if dur_cnt == 0:
                    dur_fp += (line_len - pred_dur_start_token) + (0 + pred_dur_end_token)
                
    for drug, duration in gold_dict.items():
        
        gold_drug_line = int(drug.split(':')[0])
        try:
            gold_drug_start_token = int(drug.split(':')[1].split()[0])
        except:
            gold_drug_start_token = int(drug.split(':')[1].split(',')[0])
        gold_drug_end_token = int(drug.split(':')[-1])
        gold_drug = range(gold_drug_start_token, gold_drug_end_token+1)
        gold_cnt = 0
            
        for drug in pred_dict.keys():
            
            pred_drug_line = int(drug.split(':')[0])
            pred_drug_start_token = int(drug.split(':')[1].split()[0])
            pred_drug_end_token = int(drug.split(':')[-1])
            pred_drug = range(pred_drug_start_token, pred_drug_end_token+1)
            overlap = set(pred_drug) & set(gold_drug)
                
            if pred_drug_line == gold_drug_line and overlap:
                drug_cnt += 1
                break
                
        if drug_cnt == 0:
            m_fn += len(gold_drug)
            
        if duration != 'du="nm"':
            gold_dur_start_line = int(duration.split(':')[0])
            gold_dur_end_line = int(duration.split(':')[1].split()[-1])
            gold_dur_start_token = int(duration.split(':')[1].split()[0])
            gold_dur_end_token = int(duration.split(':')[-1])
            dur_cnt = 0
            
            if gold_dur_start_line == gold_dur_end_line:
                gold_dur = range(gold_dur_start_token, gold_dur_end_token)
                for duration in pred_dict.values():
                    if duration.strip() != 'du="nm"':
                        pred_dur_line = int(duration.split(':')[0])
                        pred_dur_start_token = int(duration.split(':')[1].split()[0])
                        pred_dur_end_token = int(duration.split(':')[-1])
                        pred_dur = range(pred_dur_start_token, pred_dur_end_token)
                        overlap = set(pred_dur) & set(gold_dur)
                
                        if pred_dur_line == gold_dur_start_line and overlap:
                            dur_cnt += 1
                            break
                
                if dur_cnt == 0:
                    dur_fn += len(gold_dur) 
                    
            else:
                for duration in pred_dict.values():
                    if duration.strip() != 'du="nm"':
                        pred_dur_start_line = int(duration.split(':')[0])
                        pred_dur_end_line = int(duration.split(':')[1].split()[-1])
                        pred_dur_start_token = int(duration.split(':')[1].split()[0])
                        pred_dur_end_token = int(duration.split(':')[-1])
                        
                        with open('./input/'+filename) as f:
                            x = f.readlines()
                            for idx, line in enumerate(x):
                                if idx+1 == int(pred_start_line):
                                    line_len = len(line.split())
                                    break
                                    
                        if pred_dur_start_line == gold_dur_start_line and pred_dur_end_line == gold_dur_end_line:
                            first_line_overlap = set(range(pred_dur_start_token, line_len)) & set(range(gold_dur_start_token, line_len))
                            last_line_overlap = set(range(0, pred_dur_end_token+1)) & set(range(0, gold_dur_end_token+1))
                            
                            if first_line_overlap or second_line_overlap:
                                dur_cnt += 1
                                break
                                
                if dur_cnt == 0:
                    dur_fn += len(gold_dur)
                                                                         
m_precision = m_tp / (m_tp + m_fp)
m_recall = m_tp / (m_tp + m_fn)
m_f1 = 2 * (m_precision * m_recall) / (m_precision + m_recall)

print("Drug precision: " + str(round(m_precision, 4)))
print("Drug recall: " + str(round(m_recall, 4)))
print("Drug F1: " + str(round(m_f1, 4)) + "\n")

dur_precision = dur_tp / (dur_tp + dur_fp)
dur_recall = dur_tp / (dur_tp + dur_fn)
dur_f1 = 2 * (dur_precision * dur_recall) / (dur_precision + dur_recall)

print("Duration precision: " + str(round(dur_precision, 4)))
print("Duration recall: " + str(round(dur_recall, 4)))
print("Duration F1: " + str(round(dur_f1, 4)) + "\n")

micro_prec = (m_tp + dur_tp) / ((m_tp+dur_tp) + (m_fp+dur_fp))
micro_rec = (m_tp + dur_tp) / ((m_tp+dur_tp) + (m_fn+dur_fn))
micro_f1 = 2 * (micro_prec * micro_rec) / (micro_prec + micro_rec)

print("Micro precision: " + str(round(micro_prec, 4)))
print("Micro recall: " + str(round(micro_rec, 4)))
print("Micro F1: " + str(round(micro_f1, 4)))

Drug precision: 0.9317
Drug recall: 0.8112
Drug F1: 0.8673

Duration precision: 0.5714
Duration recall: 0.129
Duration F1: 0.2105

Micro precision: 0.9239
Micro recall: 0.7579
Micro F1: 0.8327


### Token matches, with Drools

In [11]:
m_tp = 0
m_fn = 0
m_fp = 0

dur_tp = 0
dur_fn = 0
dur_fp = 0

for filename in output_with_drools:
    
    pred_dict = {} # predicted medication index: related predicted duration index
    gold_dict = {} # gold medication index: related gold duration index
    
    with open("./output_with_drools_i2b2/"+filename) as m, open("./human_annotations/"+filename) as h:
        
        machine_lines = m.readlines()
        human_lines = h.readlines()
        
    for line in machine_lines:
            
        m = line.split('||')[0]
        dur = line.split('||')[1]
        m_idx = ' '.join(m.split()[-2:])
            
        if dur.lower().strip() == 'du="nm"':
            pred_dict[m_idx] = 'du="nm"' 
        else:
            dur_idx = ' '.join(dur.split()[-2:])
            pred_dict[m_idx] = dur_idx
                    
    for line in human_lines:
            
        m = line.split('||')[0]
        dur = line.split('||')[1]
        m_idx = ' '.join(m.split()[-2:])
            
        if dur.lower().split() == 'du="nm"':
            gold_dict[m_idx] = 'du="nm"'
            
        else:
            dur_idx = ' '.join(dur.split()[-2:])
            gold_dict[m_idx] = dur_idx
                                
    for drug, duration in pred_dict.items(): 
        pred_drug_line = int(drug.split(':')[0])
        pred_drug_start_token = int(drug.split(':')[1].split()[0])
        pred_drug_end_token = int(drug.split(':')[-1])
        pred_drug = range(pred_drug_start_token, pred_drug_end_token+1)
        drug_cnt = 0 # turns to 1 if a corresponding drug was found in the human annotations
            
        for drug in gold_dict.keys():
            gold_drug_line = int(drug.split(':')[0])
            try:
                gold_drug_start_token = int(drug.split(':')[1].split()[0])
            except: # one exception found were drug = "217:1,217:3 217:3" instead of "217:1 217:3"
                gold_drug_start_token = int(drug.split(':')[1].split(',')[0])       
            gold_drug_end_token = int(drug.split(':')[-1])
            gold_drug = range(gold_drug_start_token, gold_drug_end_token+1)
            overlap = set(pred_drug) & set(gold_drug)
                
            if pred_drug_line == gold_drug_line and overlap: 
                # corresponding drug mention found, assuming that drug mentions are not split over lines
                
                drug_cnt += 1
                m_tp += len(overlap)
                
                if pred_drug_end_token > gold_drug_end_token:
                    m_fp += pred_drug_end_token - gold_drug_end_token
                if pred_drug_end_token < gold_drug_end_token:
                    m_fn += gold_drug_end_token - pred_drug_end_token
                if pred_drug_start_token < gold_drug_start_token:
                    m_fp += gold_drug_start_token - pred_drug_start_token
                if pred_drug_start_token > gold_drug_start_token:
                    m_fn += pred_drug_start_token - gold_drug_start_token
                    
                break
                    
        if drug_cnt == 0:
            m_fp += pred_drug_end_token - pred_drug_start_token + 1
                
        if duration.strip() != 'du="nm"':
            pred_start_line = int(duration.split(':')[0])
            pred_end_line = int(duration.split(':')[1].split()[-1])
            pred_dur_start_token = int(duration.split(':')[1].split()[0])
            pred_dur_end_token = int(duration.split(':')[-1])
            dur_cnt = 0 # turns to 1 if a corresponding duration was found in the human annotations
            
            if pred_start_line == pred_end_line:
                pred_dur = range(pred_dur_start_token, pred_dur_end_token+1)
                
                for duration in gold_dict.values():
                    if duration.strip() != 'du="nm"':
                        gold_dur_line = int(duration.split(':')[0])
                        gold_dur_start_token = int(duration.split(':')[1].split()[0])
                        gold_dur_end_token = int(duration.split(':')[-1])
                        gold_dur = range(gold_dur_start_token, gold_dur_end_token+1)
                        overlap = set(pred_dur) & set(gold_dur)
                
                        if pred_start_line == gold_dur_line and overlap:
                            dur_cnt += 1
                            dur_tp += len(overlap)
                            
                            if pred_dur_end_token > gold_dur_end_token:
                                dur_fp += pred_dur_end_token - gold_dur_end_token
                            if pred_dur_end_token < gold_dur_end_token:
                                dur_fn += gold_dur_end_token - pred_dur_end_token
                            if pred_dur_start_token < gold_dur_start_token:
                                dur_fp += gold_dur_start_token - pred_dur_start_token
                            if pred_dur_start_token > gold_dur_start_token:
                                dur_fn += pred_dur_start_token - gold_dur_start_token
                            break
                
                if dur_cnt == 0:
                    dur_fp += pred_dur_end_token - pred_dur_start_token + 1
                            
            else:  
                with open('./input/'+filename) as f:
                    x = f.readlines()
                    for idx, line in enumerate(x):
                        if idx+1 == int(pred_start_line):
                            line_len = len(line.split())
                            break
                            
                for duration in gold_dict.values():
                     if duration.strip() != 'du="nm"':
                        gold_start_line = int(duration.split(':')[0])
                        gold_end_line = int(duration.split(':')[1].split()[-1])
                        gold_dur_start_token = int(duration.split(':')[1].split()[0])
                        gold_dur_end_token = int(duration.split(':')[-1])
                                    
                        if pred_start_line == gold_start_line and pred_end_line == gold_end_line:
                            first_line_overlap = set(range(pred_dur_start_token, line_len)) & set(range(gold_dur_start_token, line_len))
                            second_line_overlap = set(range(0, pred_dur_end_token+1)) & set(range(0, gold_dur_end_token+1))
                            
                            if first_line_overlap or second_line_overlap:
                                dur_cnt += 1
                                dur_tp += len(first_line_overlap)+len(second_line_overlap)
                                
                                if pred_dur_end_token > gold_dur_end_token:
                                    dur_fp += pred_dur_end_token - gold_dur_end_token
                                if pred_dur_end_token < gold_dur_end_token:
                                    dur_fn += gold_dur_end_token - pred_dur_end_token
                                if pred_dur_start_token < gold_dur_start_token:
                                    dur_fp += gold_dur_start_token - pred_dur_start_token
                                if pred_dur_start_token > gold_dur_start_token:
                                    dur_fn += pred_dur_start_token - gold_dur_start_token
                                break 
                        
                if dur_cnt == 0:
                    dur_fp += (line_len - pred_dur_start_token) + (0 + pred_dur_end_token)
                
    for drug, duration in gold_dict.items():
        
        gold_drug_line = int(drug.split(':')[0])
        try:
            gold_drug_start_token = int(drug.split(':')[1].split()[0])
        except:
            gold_drug_start_token = int(drug.split(':')[1].split(',')[0])
        gold_drug_end_token = int(drug.split(':')[-1])
        gold_drug = range(gold_drug_start_token, gold_drug_end_token+1)
        gold_cnt = 0
            
        for drug in pred_dict.keys():
            
            pred_drug_line = int(drug.split(':')[0])
            pred_drug_start_token = int(drug.split(':')[1].split()[0])
            pred_drug_end_token = int(drug.split(':')[-1])
            pred_drug = range(pred_drug_start_token, pred_drug_end_token+1)
            overlap = set(pred_drug) & set(gold_drug)
                
            if pred_drug_line == gold_drug_line and overlap:
                drug_cnt += 1
                break
                
        if drug_cnt == 0:
            m_fn += len(gold_drug)
            
        if duration != 'du="nm"':
            gold_dur_start_line = int(duration.split(':')[0])
            gold_dur_end_line = int(duration.split(':')[1].split()[-1])
            gold_dur_start_token = int(duration.split(':')[1].split()[0])
            gold_dur_end_token = int(duration.split(':')[-1])
            dur_cnt = 0
            
            if gold_dur_start_line == gold_dur_end_line:
                gold_dur = range(gold_dur_start_token, gold_dur_end_token)
                for duration in pred_dict.values():
                    if duration.strip() != 'du="nm"':
                        pred_dur_line = int(duration.split(':')[0])
                        pred_dur_start_token = int(duration.split(':')[1].split()[0])
                        pred_dur_end_token = int(duration.split(':')[-1])
                        pred_dur = range(pred_dur_start_token, pred_dur_end_token)
                        overlap = set(pred_dur) & set(gold_dur)
                
                        if pred_dur_line == gold_dur_start_line and overlap:
                            dur_cnt += 1
                            break
                
                if dur_cnt == 0:
                    dur_fn += len(gold_dur) 
                    
            else:
                for duration in pred_dict.values():
                    if duration.strip() != 'du="nm"':
                        pred_dur_start_line = int(duration.split(':')[0])
                        pred_dur_end_line = int(duration.split(':')[1].split()[-1])
                        pred_dur_start_token = int(duration.split(':')[1].split()[0])
                        pred_dur_end_token = int(duration.split(':')[-1])
                        
                        with open('./input/'+filename) as f:
                            x = f.readlines()
                            for idx, line in enumerate(x):
                                if idx+1 == int(pred_start_line):
                                    line_len = len(line.split())
                                    break
                                    
                        if pred_dur_start_line == gold_dur_start_line and pred_dur_end_line == gold_dur_end_line:
                            first_line_overlap = set(range(pred_dur_start_token, line_len)) & set(range(gold_dur_start_token, line_len))
                            last_line_overlap = set(range(0, pred_dur_end_token+1)) & set(range(0, gold_dur_end_token+1))
                            
                            if first_line_overlap or second_line_overlap:
                                dur_cnt += 1
                                break
                                
                if dur_cnt == 0:
                    dur_fn += len(gold_dur)
                                                                         
m_precision = m_tp / (m_tp + m_fp)
m_recall = m_tp / (m_tp + m_fn)
m_f1 = 2 * (m_precision * m_recall) / (m_precision + m_recall)

print("Drug precision: " + str(round(m_precision, 4)))
print("Drug recall: " + str(round(m_recall, 4)))
print("Drug F1: " + str(round(m_f1, 4)) + "\n")

dur_precision = dur_tp / (dur_tp + dur_fp)
dur_recall = dur_tp / (dur_tp + dur_fn)
dur_f1 = 2 * (dur_precision * dur_recall) / (dur_precision + dur_recall)

print("Duration precision: " + str(round(dur_precision, 4)))
print("Duration recall: " + str(round(dur_recall, 4)))
print("Duration F1: " + str(round(dur_f1, 4)) + "\n")

micro_prec = (m_tp + dur_tp) / ((m_tp+dur_tp) + (m_fp+dur_fp))
micro_rec = (m_tp + dur_tp) / ((m_tp+dur_tp) + (m_fn+dur_fn))
micro_f1 = 2 * (micro_prec * micro_rec) / (micro_prec + micro_rec)

print("Micro precision: " + str(round(micro_prec, 4)))
print("Micro recall: " + str(round(micro_rec, 4)))
print("Micro F1: " + str(round(micro_f1, 4)))

Drug precision: 0.9448
Drug recall: 0.818
Drug F1: 0.8768

Duration precision: 0.4706
Duration recall: 0.2254
Duration F1: 0.3048

Micro precision: 0.92
Micro recall: 0.7642
Micro F1: 0.8349
